<a href="https://colab.research.google.com/github/finiteautomata/lingcomp/blob/master/Taller.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Detección de regionalismos en Twitter

En esta notebook, mostraremos cómo calcular regionalismos en base a tweets extraídos de las provincias argentinas.

Si bien todas las palabras que encontraremos no son necesariamente regionalismos, sí trataremos de encontrar palabras que guarden alguna correlación geográfica: por ejemplo, nombres de políticos, topónimos (ciudades, lugares, etc)

## Tecnologías que usaremos

Para ello utilizaremos el entorno de desarrollo Google Colab, que se basa en el lenguaje `Python`  y Jupyter.




## Cargar los datos
En primer lugar, carguemos los datos. Esto lo haremos con la librería `pandas` que, a grandes rasgos, nos permite trabajar con datos tabulados de una manera muy sencilla. 

Cada "fila" de esta tabla representa a un usuario de la muestra que tenemos. 




In [0]:

import pandas as pd


df = pd.read_json("https://raw.githubusercontent.com/finiteautomata/lingcomp/master/data/users_small.json")
print("Tenemos {} usuarios".format(len(df)))
df[:10]

Tenemos 1000 usuarios


,provincia,text
1001198857,tierradelfuego,Que lindo fue escucharla hoy a mi hija decir '...
1006781437,tierradelfuego,Que cagaso me da ir a entrenar con estos rayos...
101534453,jujuy,#Jujuy: encuentran a toda una familia muerta e...
1015480064,rionegro,Que embole chabonnn\nNunca me subestimes aaa\n...
1017761377,formosa,Me hace tanto bien estar con mis amigos!!!\nat...
1019018742,formosa,Cuántos segundos tarda una sonrisa nuestra en ...
1019973290,mendoza,Quiero ir al cine 🙌\nMientras exista un 1% de ...
1024515116,sanjuan,"Nos pusieron aire, no damos mas de felicess🙌🙌 ..."
1025513040,tierradelfuego,Que lindo tener a fer en mi vida♡\nEstoy muy c...
1029563737,neuquen,"Que bien el barca, mucho huevo\nConfió en vos...."


Calculemos las provincias que vienen en el dataset

In [0]:

provincias = df["provincia"].unique()

print("Tabla con cantidad de usuarios por provincia")
df.groupby("provincia").count()

Tabla con cantidad de usuarios por provincia


,text
provincia,
buenosaires,42
catamarca,49
chaco,38
chubut,46
cordoba,40
corrientes,47
entrerios,42
formosa,48
jujuy,42



Veamos un poco mejor qué información tiene cada usuario:



In [0]:
def show_info(user_number):
  row = df.iloc[user_number]
  print("Usuario de {}\n\n".format(row["provincia"]))
  
  print("Algunos tweets:\n")
  
  tweets = row["text"].split("\n")
  
  for (i, tweet) in zip(range(40), tweets):
      print(tweet)


#
# si cambiamos el 77 por cualquier número entre 0 y 999 y ejecutamos (shift+enter)
# podremos ver qué información de cada usuario tenemos
show_info(99)

Usuario de sanjuan


Algunos tweets:

@perroscalle todos a Irlandaaa!
@Tato_Carrizo dale dale dale!, estudia a fondooooo y despues t venis a brindar tranquilo!!!
Vamos que aca estan los estilos de Donata en el Tap Take Over del Martes 14 y Miercoles 15 de marzo. Saint Patricks… https://t.co/VqfnKDLXIy
Dale dale y dale! https://t.co/10Yp4vqv5n
https://t.co/VwnmMmIMo3
Te la vas a perder? Pinta de coleccion con cerveza estilo a eleccion + una truckers de Leinster = $300.- Nada nadit… https://t.co/OrQw1SOESa
Se viene se viene! https://t.co/xRR04UflEn
Comienzan los preparativos para su nuevo Video Clip d @batidodecoco_ok !, les abrimos la puerta para q usen las instalaciones del Irlandes ☘
#TapTakeOver #SaintPatricksDay #BarIrlandes #LeinsterBar 16;17 y 18 de Marzo EXCLUSIVIDAD de Cerveza "Ancestral" (t… https://t.co/55YbIefHCJ
#TapTakeOver #SaintPatricksDay #BarIrlandes #LeinsterBar Martes 14 y Miercoles 15 de Marzo EXCLUSIVIDAD de Cerveza… https://t.co/d2XrINxH3g
Dale Leinster!, la hincha

## Tokenización

Para poder analizar el léxico de cada tweet, necesitamos separarlos en "tokens" – es decir, en las unidades que nosotros consideremos. Este proceso se llama "tokenización", y para ello utilizaremos la librería [nltk](https://www.nltk.org/) (Natural Language ToolKit), nuestra navaja suiza para procesar texto. 

In [0]:
import re
from nltk.tokenize import TweetTokenizer

_tokenizer = TweetTokenizer(
    preserve_case=False,
    reduce_len=True,
    strip_handles=True
)

urls = r'(?:https?\://t.co/[\w]+)'


def tokenize(text, only_alpha=True, remove_hashtags=True):
    """
    Tokenize tweet
    """
    
    tokens = _tokenizer.tokenize(text)

    if only_alpha:
        tokens = [tk for tk in tokens if tk.isalpha()]
    else:
        if remove_hashtags:
            tokens = [tk for tk in tokens if tk[0] != "#"]
        tokens = [tk for tk in tokens if not re.match(urls, tk)]
    return tokens

  
tokenize("Hola a TODOSSS!!! http://www.google.com.ar 😄😄")

['hola', 'a', 'todosss']

Ahora, armemos una tabla de doble entrada de las palabras a la cantidad de veces (y de usuarios) que tienen por provincia

In [0]:
%%time

word_to_tweets = {}

def build_dataframe_from_users(users):
    """
    Build dataframe from users.

    Arguments:
    ---------

    users: Iterable of dicts having "text" key

    Returns:
    --------

    df: pandas.DataFrame

        DataFrame of occurrences of words in provinces
    """

    def get_bow(user):
        bow = {}
        province = user["provincia"]
        tweets = user["text"].split("\n")
        
        for tweet in tweets:
            tokens = tokenize(tweet)
            for token in tokens:
                if token not in word_to_tweets:
                    word_to_tweets[token] = []
                
                word_to_tweets[token].append((province, tweet))
                bow[token] = bow.get(token, 0) + 1
        return bow

    columns = {}

    for user in users:
        bow = get_bow(user)
        province = user["provincia"]

        occ_column = province + "_ocurrencias"
        user_column = province + "_usuarios"

        if occ_column not in columns:
            columns[occ_column] = {}
            columns[user_column] = {}

        for tok, count in bow.items():
            columns[occ_column][tok] = columns[occ_column].get(tok, 0) + count
            columns[user_column][tok] = columns[user_column].get(tok, 0) + 1


    df = pd.DataFrame.from_dict(columns)

    df.fillna(0, inplace=True)
    return df
  

word_df = build_dataframe_from_users(row for index, row in df.iterrows())

CPU times: user 1min 52s, sys: 66.9 ms, total: 1min 53s
Wall time: 1min 53s


Veamos cómo se ve esto

In [0]:
word_df[:100]

,buenosaires_ocurrencias,buenosaires_usuarios,catamarca_ocurrencias,catamarca_usuarios,chaco_ocurrencias,chaco_usuarios,chubut_ocurrencias,chubut_usuarios,cordoba_ocurrencias,cordoba_usuarios,...,santacruz_ocurrencias,santacruz_usuarios,santafe_ocurrencias,santafe_usuarios,santiago_ocurrencias,santiago_usuarios,tierradelfuego_ocurrencias,tierradelfuego_usuarios,tucuman_ocurrencias,tucuman_usuarios
a,11153.0,40.0,13164.0,49.0,9215.0,36.0,15540.0,46.0,11744.0,40.0,...,10858.0,39.0,12751.0,40.0,13278.0,44.0,14897.0,50.0,12495.0,40.0
aa,15.0,6.0,10.0,5.0,1.0,1.0,43.0,14.0,7.0,4.0,...,13.0,8.0,10.0,7.0,5.0,3.0,16.0,11.0,9.0,6.0
aaa,32.0,11.0,27.0,10.0,12.0,4.0,191.0,22.0,47.0,13.0,...,11.0,8.0,16.0,7.0,26.0,8.0,72.0,15.0,14.0,7.0
aaaa,11.0,7.0,12.0,4.0,11.0,7.0,49.0,13.0,6.0,6.0,...,5.0,4.0,5.0,5.0,1.0,1.0,16.0,7.0,4.0,3.0
aaaaa,2.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
aaaaaa,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
aaaaaaa,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
aaaaaaaa,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
aaaaaaaaa,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
aaaaaaaaaa,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
columnas_palabras = ["{}_ocurrencias".format(prov) for prov in provincias]
columnas_usuarios = ["{}_usuarios".format(prov) for prov in provincias]

word_df["cantidad_palabras"] = word_df[columnas_palabras].sum(axis=1)
word_df["cantidad_usuarios"] = word_df[columnas_usuarios].sum(axis=1)


len(columnas_palabras), len(columnas_usuarios)

(23, 23)

# Filtrado de palabras

Saquemos las palabras que ocurren muy pocas veces (menos de 10 ocurrencias, menos de 2 usuarios)

In [0]:
total_word_df = word_df

word_df = word_df[(word_df["cantidad_palabras"] >= 10) & (word_df["cantidad_usuarios"] >= 3)].copy()


word_df.shape

(33308, 48)

## Cálculo de la métrica

Primero, calculemos los logaritmos de las cantidades de usuarios y palabras, y la entropía de las ocurrencias y de usuarios.

In [0]:
import numpy as np
from scipy.stats import entropy

word_df["log_usuarios"] = np.log(word_df["cantidad_usuarios"])
word_df["log_palabras"] = np.log(word_df["cantidad_palabras"])

word_df["entropia_palabras"] = word_df[columnas_palabras].apply(entropy, axis=1, raw=True)
word_df["entropia_usuarios"] = word_df[columnas_usuarios].apply(entropy, axis=1, raw=True)



Calculemos la métrica. Recordemos que su fórmula era


$$ 
IV(w) =  log(f(w)) * (M - H(p|w))
$$

donde $log(f(w))$ es el logaritmo de la cantidad de veces que aparece la palabra en cada provincia o la cantidad de usuarios,  $M$ es la entropía máxima ($log(23)$) y $H(p|w)$ es la entropía de la palabra (bien para ocurrencias a secas, o bien para usuarios).

Para escalarlos, normalizamos entre 0 y 1 el logaritmo dividiendo por el máximo.

In [0]:

log_tf_palabras = word_df["log_palabras"] / word_df["log_palabras"].max()
log_tf_usuarios = word_df["log_usuarios"] / word_df["log_usuarios"].max()


word_df["metrica_palabras"] = log_tf_palabras * (np.log(23) - word_df["entropia_palabras"])
word_df["metrica_usuarios"] = log_tf_usuarios * (np.log(23) - word_df["entropia_usuarios"])

Ahora, ordenemos todo por la métrica de usuarios!

In [0]:

word_df.sort_values("metrica_usuarios", ascending=False, inplace=True)

Veamos las 100 primeras palabras de la métrica

Obs: si cambiamos `range(0, 100)` por `range(a, b)` podremos ver todas las palabras entre las posiciones a y b del listado

In [0]:

def mostrar_palabras(desde, hasta):
  per_line = 4

  for i in range(desde, hasta+1):
    print("{:<4} -- {:<16}".format(i+1, word_df.index[i]), end='')
    if (i+1) % per_line == 0:
      print('')

mostrar_palabras(0, 200)

1    -- ush             2    -- fsa             3    -- rada            4    -- argel           
5    -- patagones       6    -- jei             7    -- obera           8    -- rg              
9    -- tolhuin         10   -- yarca           11   -- poec            12   -- chayero         
13   -- riojano         14   -- riojanos        15   -- viedma          16   -- oberá           
17   -- malpegue        18   -- sanagasta       19   -- toay            20   -- bragado         
21   -- riojana         22   -- nae             23   -- ctes            24   -- reviro          
25   -- chilecito       26   -- qlq             27   -- pachata         28   -- chimbas         
29   -- cldo            30   -- nqn             31   -- mbae            32   -- posadas         
33   -- beder           34   -- nox             35   -- riojanas        36   -- blv             
37   -- aijue           38   -- edelar          39   -- chayar          40   -- anga            
41   -- músicas         42   -

In [0]:
def info_palabra(palabra):
  """
  Información de la palabra dada.
  
  Argumentos:
  -----------
  
  palabra: string
    Una palabra perteneciente al vocabulario empleado
  """
  row = word_df.loc[palabra]
  
  print(palabra)
  
  user_row = row[columnas_usuarios]

  user_row = user_row.sort_values(ascending=False)
  
  
  
  print("Provincias donde más se usa:")
  
  for (column, value) in user_row.iteritems():
      if value > 0:
          provincia = column.split("_")[0]
          print("{:<15} -- {:<2} usuarios".format(provincia, int(value)))
          
  print("\n\nAlgunos ejemplos:")
  
  for (i, tweet) in zip(range(20), word_to_tweets[palabra]):
      print(" {:<16} -- {}".format(tweet[0], tweet[1]))
  

info_palabra("asada")

asada
Provincias donde más se usa:
sanjuan         -- 13 usuarios
mendoza         -- 12 usuarios
corrientes      -- 3  usuarios
santafe         -- 1  usuarios
santiago        -- 1  usuarios
sanluis         -- 1  usuarios


Algunos ejemplos:
 corrientes       -- El día amerita un buen asada de medio dia
 mendoza          -- Ya fue, estoy asada
 mendoza          -- Estoy asada
 sanjuan          -- Que asada con instagram 😭😭😭
 sanjuan          -- Que asada estoy 😣😣
 mendoza          -- Estoy asada y con ganas de romper todo. Me pasa por pelotudaaaaaaáaaaa a a
 sanjuan          -- estoy muy muy asada 🔫
 sanjuan          -- Que asada que estoy, como me van a cambiar 🔫
 sanjuan          -- tan asada puedo estar?
 sanjuan          -- Entre que estoy asada con la minita y me llegan las notificaciones de sus tw quiero largar el teléfono por la ventana...
 sanjuan          -- Vengo cínica, fobica, cruda, hervida y asada por vos..
 sanjuan          -- No puedo estar tan asada.
 sanjuan          -